In [30]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,14 mins 49 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_paaske_2eiu45
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.639 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [31]:
# Reading csv to h2o dataframe 
col_types_train = {
 'chain_name': 'enum',
 'mall_name': 'enum',
 'revenue': 'numeric',
 #'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'numeric',
 #'counts_gr_lv2': 'numeric',
 #'counts_municipality_lv2': 'numeric',
 #'busstops_within_50m': 'numeric',
 #'busstops_within_100m': 'numeric',
 #'busstops_within_400m': 'numeric',
 #'busstops_within_800m': 'numeric',
 'busstops_within_1500m': 'numeric',
 #'num_stores_within_100m_and_same_lvl2': 'numeric',
 'num_stores_within_500m_and_same_lvl2': 'numeric',
 #'num_stores_within_1km_and_same_lvl2': 'numeric',
 #'num_stores_within_5km_and_same_lvl2': 'numeric',
 #'num_stores_within_10km_and_same_lvl2': 'numeric',
 'num_stores_within_20km_and_same_lvl2': 'numeric',
 'closest_competitor_lv2': 'numeric',
 'mean_revenue_lv3' : 'numeric',
 'mean_revenue_lv2' : 'numeric',
 'median_revenue_lv3' : 'numeric',
 'median_revenue_lv2' : 'numeric',
 'mean_revenue_chain' : 'numeric',
 'median_revenue_lv4' : 'numeric',
 'mean_revenue_lv4' : 'numeric'
 #'num_stores_within_100m_and_same_lvl3': 'numeric',
 #'num_stores_within_500m_and_same_lvl3': 'numeric',
 #'num_stores_within_1km_and_same_lvl3': 'numeric',
 #'num_stores_within_5km_and_same_lvl3': 'numeric',
 #'num_stores_within_10km_and_same_lvl3': 'numeric',
 #'num_stores_within_20km_and_same_lvl3': 'numeric',
 #'closest_competitor_lv3': 'numeric',
 #'municipality_size_group': 'enum',
 #'mean_revenue_for_municipality_size_group': 'numeric',
 #'median_revenue_for_municipality_size_group': 'numeric',
 #'st_dev_of_revenue_for_municipality_size_group': 'numeric',
 #'municipality_rev_group_lv2': 'numeric',
 #'mean_revenue_for_municipality_rev_group_lv2': 'numeric'
 }

col_types_test = {
 'chain_name': 'enum',
 'mall_name': 'enum',
 #'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'numeric',
 #'counts_gr_lv2': 'numeric',
 #'counts_municipality_lv2': 'numeric',
 #'busstops_within_50m': 'numeric',
 #'busstops_within_100m': 'numeric',
 #'busstops_within_400m': 'numeric',
 #'busstops_within_800m': 'numeric',
 'busstops_within_1500m': 'numeric',
 #'num_stores_within_100m_and_same_lvl2': 'numeric',
 'num_stores_within_500m_and_same_lvl2': 'numeric',
 #'num_stores_within_1km_and_same_lvl2': 'numeric',
 #'num_stores_within_5km_and_same_lvl2': 'numeric',
 #'num_stores_within_10km_and_same_lvl2': 'numeric',
 'num_stores_within_20km_and_same_lvl2': 'numeric',
 'closest_competitor_lv2': 'numeric',
 'mean_revenue_lv3' : 'numeric',
 'mean_revenue_lv2' : 'numeric',
 'median_revenue_lv3' : 'numeric',
 'median_revenue_lv2' : 'numeric',
 'mean_revenue_chain' : 'numeric',
 'median_revenue_lv4' : 'numeric',
 'mean_revenue_lv4' : 'numeric'
 #'num_stores_within_100m_and_same_lvl3': 'numeric',
 #'num_stores_within_500m_and_same_lvl3': 'numeric',
 #'num_stores_within_1km_and_same_lvl3': 'numeric',
 #'num_stores_within_5km_and_same_lvl3': 'numeric',
 #'num_stores_within_10km_and_same_lvl3': 'numeric',
 #'num_stores_within_20km_and_same_lvl3': 'numeric',
 #'closest_competitor_lv3': 'numeric',
 #'municipality_size_group': 'enum',
 #'mean_revenue_for_municipality_size_group': 'numeric',
 #'median_revenue_for_municipality_size_group': 'numeric',
 #'st_dev_of_revenue_for_municipality_size_group': 'numeric',
 #'municipality_rev_group_lv2': 'numeric',
 #'mean_revenue_for_municipality_rev_group_lv2': 'numeric'
 }

train_data = pd.read_csv('old_data/feature_data/training_set_dropped_old.csv')
test_data = pd.read_csv('old_data/feature_data/testing_set_dropped_old.csv')

# Save store ids, then remove this column
store_ids = test_data['store_id'].to_numpy()
ids = pd.DataFrame(store_ids)
ids.rename(columns={0 :'id'}, inplace=True)

train_data = train_data.drop('store_id', axis=1)
test_data = test_data.drop('store_id', axis=1)

train_data.to_csv('feature_data/training_set_dropped.csv', index=False)
test_data.to_csv('feature_data/testing_set_dropped.csv', index=False)

train_data = h2o.import_file('feature_data/training_set_dropped.csv', col_types = col_types_train)
test_data = h2o.import_file('feature_data/testing_set_dropped.csv', col_types = col_types_test)

test_data.types





KeyError: 'store_id'

In [ ]:
# Removing labels from train set
y = "revenue"
x = train_data.columns
x.remove(y)
test_data

In [ ]:
# Running fit/training on train set with cross validation n=5
aml = H2OAutoML(max_models = 15, seed = 2, stopping_metric = "RMSE", exclude_algos=["XGBoost", "DeepLearning"])
aml.train(x = x, y = y, training_frame = train_data)

In [ ]:
lb = aml.leaderboard

In [ ]:
# Showing best perforimg models
lb.head()

In [ ]:
preds = aml.predict(test_data)

In [ ]:
#print variable importance
corr = aml.varimp(use_pandas = True)
corr['sum'] = corr.sum(axis=1)
corr = corr.sort_values(by=['sum'])
corr.head(30)

In [ ]:
#preds = h2o.as_list(preds)
print(preds)

In [ ]:
# Convert result to desired format

import pandas as pd
import numpy as np


pandas_preds = preds.as_data_frame()
pandas_preds.rename(columns={'predict' :'predicted'}, inplace=True)
pandas_preds['predicted'] = np.expm1(pandas_preds['predicted'])

def save_submission(pred):
    predicted = pd.DataFrame(pred, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.to_csv('h2o.csv', index=False)
    
save_submission(pandas_preds)



In [ ]:
aml.explain(frame = train_data)